In [1]:
import matplotlib
import bokeh
import plotly
import sklearn

In [2]:
import sparkhpc
from sparkhpc import sparkjob
import findspark 
findspark.init("/cluster/home/simondi/spark/")
import pyspark


In [3]:
from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType

In [4]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector

In [5]:
sc = pyspark.SparkContext("local", "App Name")

In [6]:
spark = pyspark.sql.SparkSession(sc)

In [7]:
file_name = "/cluster/home/simondi/simondi/tix/data/screening_data/cells_sample_10.tsv"

In [8]:
df = spark.read.csv(path=file_name, sep="\t", header='true')

In [9]:
df.cache()

DataFrame[study: string, pathogen: string, library: string, design: string, replicate: string, plate: string, well: string, gene: string, sirna: string, well_type: string, image_idx: string, object_idx: string, cells.areashape_area: string, cells.areashape_eccentricity: string, cells.areashape_extent: string, cells.areashape_formfactor: string, cells.areashape_majoraxislength: string, cells.areashape_minoraxislength: string, cells.areashape_perimeter: string, cells.children_bacteria_count: string, cells.children_invasomes_count: string, cells.location_center_x: string, cells.location_center_y: string, cells.neighbors_anglebetweenneighbors_2: string, cells.neighbors_firstclosestobjectnumber_2: string, cells.neighbors_firstclosestxvector_2: string, cells.neighbors_firstclosestyvector_2: string, cells.neighbors_numberofneighbors_2: string, cells.neighbors_secondclosestobjectnumber_2: string, cells.neighbors_secondclosestxvector_2: string, cells.neighbors_secondclosestyvector_2: string, ce

In [10]:
old_cols = df.schema.names
new_cols = list(map(lambda x: x.replace(".", "_"),  old_cols))
df = reduce(lambda data, idx: data.withColumnRenamed(old_cols[idx], new_cols[idx]), range(len(new_cols)), df)
for i, x in enumerate(new_cols):
    if x.startswith("cells"):
        df= df.withColumn(x, df[x].cast("double"))

In [11]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[x for x in new_cols if x.startswith("cells")], outputCol='features')

In [12]:
X = assembler.transform(df)

In [13]:
Y = X.limit(100)
Y.cache()

DataFrame[study: string, pathogen: string, library: string, design: string, replicate: string, plate: string, well: string, gene: string, sirna: string, well_type: string, image_idx: string, object_idx: string, cells_areashape_area: double, cells_areashape_eccentricity: double, cells_areashape_extent: double, cells_areashape_formfactor: double, cells_areashape_majoraxislength: double, cells_areashape_minoraxislength: double, cells_areashape_perimeter: double, cells_children_bacteria_count: double, cells_children_invasomes_count: double, cells_location_center_x: double, cells_location_center_y: double, cells_neighbors_anglebetweenneighbors_2: double, cells_neighbors_firstclosestobjectnumber_2: double, cells_neighbors_firstclosestxvector_2: double, cells_neighbors_firstclosestyvector_2: double, cells_neighbors_numberofneighbors_2: double, cells_neighbors_secondclosestobjectnumber_2: double, cells_neighbors_secondclosestxvector_2: double, cells_neighbors_secondclosestyvector_2: double, ce

In [14]:
kmeans = KMeans(k=5, seed=23)
model = kmeans.fit(Y)

In [15]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

In [16]:
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
pca_model = pca.fit(Y)

In [17]:
result = pca_model.transform(Y).select("pcaFeatures")

In [20]:
import matplotlib
import matplotlib.pyplot 

x = [1,2,3,4]
y = [3,4,8,6]

matplotlib.s.scatter(x,y)

matplotlib.pyplot.show()

AttributeError: 'str' object has no attribute 'scatter'